In [1]:
# Import packages
import numpy as np
import pandas as pd
import scipy.stats as sts
import os
import matplotlib.pyplot as plt
import scipy.optimize as opt
import warnings
warnings.filterwarnings('ignore')
from scipy import special
from scipy.integrate import quad

In [2]:
# Load data
data = np.loadtxt('MacroSeries.txt', delimiter=",")
c = data[:, 0]
k = data[:, 1]
w = data[:, 2]
r = data[:, 3]

In [3]:
def get_z(r,k,alpha):
    z = np.log(r) - np.log(alpha) - (alpha - 1) * np.log(k)
    return z
    

In [4]:
def data_moments(c, k, w, r, alpha, beta, rho, mu):
    z = get_z(r, k, alpha)
    eq7 = z[1:] - rho * z[:-1] - (1 - rho) * mu
    eq8 = (z[1:] - rho * z[:-1] - (1 - rho) * mu) * z[:-1]
    eq9 = beta * alpha * np.exp(z[1:]) * k[1:] ** (alpha - 1) * c[:-1]/c[1:] - 1
    eq10 = (beta * alpha * np.exp(z[1:]) * k[1:] ** (alpha - 1) * c[:-1]/c[1:] - 1) * w[:-1]
    
    return eq7.mean(), eq8.mean(), eq9.mean(), eq10.mean()

def err_vec(c, k, w, r, alpha, beta, rho, mu, simple = True):
    mo_moments = np.zeros(4)
    da_moments = np.array(data_moments(c, k, w, r, alpha, beta, rho, mu)).reshape(1, 4)
    if simple:
        err_vec = (mo_moments - da_moments)[0]
    else:
        err_vec = (mo_moments - da_moments) / da_moments
    return err_vec

def crit(params, *args):
    alpha, beta, rho, mu = params
    c, k, w, r, weight  = args
    err = err_vec(c, k, w, r, alpha, beta, rho, mu)
    crit_val = np.dot(np.dot(err.T, weight), err) 
    
    return crit_val

In [5]:
weight = np.identity(4)
alpha_init = 0.4
beta_init = 0.8
rho_init = 0.7
mu_init = 9

params_init = (alpha_init, beta_init, rho_init, mu_init)
bnds = ((1e-5, 1-1e-5), (1e-5, 1-1e-5), (-1+1e-5, 1-1e-5), (1e-5, None))
args = (c, k, w, r, weight)
results = opt.minimize(crit, params_init, args=(args), bounds = bnds, method = 'L-BFGS-B')
print(results)

      fun: 0.002171727328676759
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([3.58156718e-02, 2.03044618e+03, 4.88148150e-05, 8.97812553e-04])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 235
      nit: 28
   status: 0
  success: True
        x: array([0.47250576, 0.99      , 0.71564474, 9.00356726])
